In [1]:
import pandas as pd
from sqlalchemy import create_engine,text

import os 
from datetime import datetime

In [2]:
data_path = os.path.join(os.getcwd(),".\sample_data")

In [3]:
os.listdir(data_path)

['superstore_orders.csv', 'superstore_returns.csv']

In [4]:
superstore_data = pd.read_csv(os.path.join(data_path,'superstore_orders.csv'),encoding = 'latin')

In [5]:
def date_alignment(parent_data,date_col):
    df = parent_data.copy()
    
    def calculate_date_difference(df, date_column1, date_column2, difference_column):
        df[date_column1] = pd.to_datetime(df[date_column1], format='%d/%m/%y')
        df[date_column2] = pd.to_datetime(df[date_column2], format='%d/%m/%y')
        df[difference_column] = (df[date_column1] - df[date_column2]).dt.days
        return df
    
    df = calculate_date_difference(df,date_col,"Order Date", "date_diff")
    
    print("actual min date :",df[date_col].min(),
          "\nactual max_date :",df[date_col].max())
    
    current_date = datetime.now().date()
    current_datetime = datetime.combine(current_date, datetime.min.time())
    current_date = pd.Timestamp(current_datetime)
    
    offset = current_date.date() - df[date_col].max().date()
    df[date_col] = df[date_col] + pd.to_timedelta(offset)
    
    print("---------------------------------------------")
    print("aligned min date :",df[date_col].min(),
          "\naligned max_date :",df[date_col].max())
    
    df['Order Date'] = df[date_col] - pd.to_timedelta(df['date_diff'], unit='d')
    df.drop(columns = ["date_diff"], inplace = True)
    
    return df

In [6]:
orders_df = date_alignment(superstore_data,"Ship Date")

actual min date : 2019-01-07 00:00:00 
actual max_date : 2023-01-05 00:00:00
---------------------------------------------
aligned min date : 2020-07-29 00:00:00 
aligned max_date : 2024-07-27 00:00:00


In [7]:
orders_df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country/Region,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,US-2019-103800,2020-07-25,2020-07-29,Standard Class,DP-13000,Darren Powers,Consumer,United States,Houston,...,77095,Central,OFF-PA-10000174,Office Supplies,Paper,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",16.448,2,0.2,5.5512
1,2,US-2019-112326,2020-07-26,2020-07-30,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-BI-10004094,Office Supplies,Binders,GBC Standard Plastic Binding Systems Combs,3.540,2,0.8,-5.4870
2,3,US-2019-112326,2020-07-26,2020-07-30,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-LA-10003223,Office Supplies,Labels,Avery 508,11.784,3,0.2,4.2717
3,4,US-2019-112326,2020-07-26,2020-07-30,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-ST-10002743,Office Supplies,Storage,SAFCO Boltless Steel Shelving,272.736,3,0.2,-64.7748
4,5,US-2019-141817,2020-07-27,2020-08-03,Standard Class,MB-18085,Mick Brown,Consumer,United States,Philadelphia,...,19143,East,OFF-AR-10003478,Office Supplies,Art,Avery Hi-Liter EverBold Pen Style Fluorescent ...,19.536,3,0.2,4.8840


In [8]:
orders_df["Ship Date"].max()

Timestamp('2024-07-27 00:00:00')

In [9]:
orders_df["Order Date"].max()

Timestamp('2024-07-21 00:00:00')

In [10]:
column_mapping = {
    'Row ID': 'row_id',
    'Order ID': 'order_id',
    'Order Date': 'order_date',
    'Ship Date': 'ship_date',
    'Ship Mode': 'ship_mode',
    'Customer ID': 'customer_id',
    'Customer Name': 'customer_name',
    'Segment': 'segment',
    'Country/Region': 'country_region',
    'City': 'city',
    'State/Province': 'state_province',
    'Postal Code': 'postal_code',
    'Region': 'region',
    'Product ID': 'product_id',
    'Category': 'category',
    'Sub-Category': 'sub_category',
    'Product Name': 'product_name',
    'Sales': 'sales',
    'Quantity': 'quantity',
    'Discount': 'discount',
    'Profit': 'profit'
}
orders_df.rename(columns=column_mapping, inplace=True)

In [11]:
orders_df.to_csv("orders_superstore_24_july_2024.csv",index = False)

In [12]:
db_host = 'localhost'
db_port = 5432
db_username = 'postgres'
db_pass = 'animesh'
db_name = 'data_store'

db_uri = f"postgresql+psycopg2://{db_username}:{db_pass}@{db_host}:{db_port}/{db_name}"


engine = create_engine(db_uri)

def replace_table_data(data, table_name):

    with engine.connect() as connection:
        with connection.begin():
            connection.execute(text(f"DELETE FROM {table_name}"))
            
            data.to_sql(table_name, connection, if_exists='append', index=False)


replace_table_data(orders_df, 'superstore')

print("Table data replaced successfully.")


Table data replaced successfully.


In [22]:
def query_table(table_name, query):
   
    with engine.connect() as connection:
        result = connection.execute(text(query))
        df = pd.DataFrame(result.fetchall(), columns=result.keys())
    
    return df


query = """

WITH yearly_sales AS (SELECT DATE_TRUNC('year', "order_date") AS year, "category", SUM("sales") AS total_sales FROM "superstore" GROUP BY year, "category") SELECT year, "category", total_sales FROM yearly_sales ORDER BY year, total_sales DESC LIMIT 100;
"""  



result_df = query_table('superstore', query)
result_df

,year,category,total_sales
0,2020-01-01 00:00:00+05:30,Technology,59623.41
1,2020-01-01 00:00:00+05:30,Furniture,48763.33
2,2020-01-01 00:00:00+05:30,Office Supplies,39436.22
3,2021-01-01 00:00:00+05:30,Furniture,164024.76
4,2021-01-01 00:00:00+05:30,Technology,161376.95
5,2021-01-01 00:00:00+05:30,Office Supplies,161149.51
6,2022-01-01 00:00:00+05:30,Technology,209915.04
7,2022-01-01 00:00:00+05:30,Furniture,179429.36
8,2022-01-01 00:00:00+05:30,Office Supplies,146410.93
9,2023-01-01 00:00:00+05:30,Technology,223110.87


In [14]:
orders_df.groupby("category").agg({"sales":sum,"order_id":"nunique"})

,sales,order_id
category,,
Furniture,754747.7613,1819
Office Supplies,731893.3140,3812
Technology,839893.2790,1560


In [15]:
754747.7613/1819

414.92455266630014

In [16]:
orders_df

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country_region,city,...,postal_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit
0,1,US-2019-103800,2020-07-25,2020-07-29,Standard Class,DP-13000,Darren Powers,Consumer,United States,Houston,...,77095,Central,OFF-PA-10000174,Office Supplies,Paper,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",16.448,2,0.2,5.5512
1,2,US-2019-112326,2020-07-26,2020-07-30,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-BI-10004094,Office Supplies,Binders,GBC Standard Plastic Binding Systems Combs,3.540,2,0.8,-5.4870
2,3,US-2019-112326,2020-07-26,2020-07-30,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-LA-10003223,Office Supplies,Labels,Avery 508,11.784,3,0.2,4.2717
3,4,US-2019-112326,2020-07-26,2020-07-30,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-ST-10002743,Office Supplies,Storage,SAFCO Boltless Steel Shelving,272.736,3,0.2,-64.7748
4,5,US-2019-141817,2020-07-27,2020-08-03,Standard Class,MB-18085,Mick Brown,Consumer,United States,Philadelphia,...,19143,East,OFF-AR-10003478,Office Supplies,Art,Avery Hi-Liter EverBold Pen Style Fluorescent ...,19.536,3,0.2,4.8840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10189,10190,US-2022-143259,2024-07-21,2024-07-25,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,New York City,...,10009,East,OFF-BI-10003684,Office Supplies,Binders,Wilson Jones Legal Size Ring Binders,52.776,3,0.2,19.7910
10190,10191,US-2022-115427,2024-07-21,2024-07-25,Standard Class,EB-13975,Erica Bern,Corporate,United States,Fairfield,...,94533,West,OFF-BI-10004632,Office Supplies,Binders,GBC Binding covers,20.720,2,0.2,6.4750
10191,10192,US-2022-156720,2024-07-21,2024-07-25,Standard Class,JM-15580,Jill Matthias,Consumer,United States,Loveland,...,80538,West,OFF-FA-10003472,Office Supplies,Fasteners,Bagged Rubber Bands,3.024,3,0.2,-0.6048
10192,10193,US-2022-143259,2024-07-21,2024-07-25,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,New York City,...,10009,East,TEC-PH-10004774,Technology,Phones,Gear Head AU3700S Headset,90.930,7,0.0,2.7279


In [ ]:
"""
SELECT
   SUM("sales") AS total_sales,
   DATE_TRUNC('week', "order_date") AS week_start
FROM "superstore"
WHERE "order_date" >= CURRENT_DATE - INTERVAL '7 days'
GROUP BY week_start

UNION ALL
SELECT 
    SUM("sales") AS total_sales,
    DATE_TRUNC('week', "order_date") AS week_start
FROM "superstore"
WHERE "order_date" >= CURRENT_DATE - INTERVAL '37 days' AND "order_date" < CURRENT_DATE - INTERVAL '30 days' 
GROUP BY week_start
ORDER BY week_start DESC 
LIMIT 100;

"""

In [ ]:
"""
SELECT 
    SUM("sales") AS total_sales, 
    DATE_TRUNC('week', "order_date") AS week_start 
FROM "superstore" 
WHERE "order_date" >= DATE_TRUNC('week', CURRENT_DATE) - INTERVAL '7 days' 
AND "order_date" < DATE_TRUNC('week', CURRENT_DATE)
GROUP BY week_start;



"""

In [25]:
orders_df.groupby("order_date").agg(total_sales = ("sales",sum)).reset_index().query("order_date == '2024-07-11'")

,order_date,total_sales
1234,2024-07-11,1926.776


In [26]:
orders_df.groupby("order_date").agg(total_sales = ("sales",sum)).reset_index().query("order_date == '2024-07-18'")

,order_date,total_sales
1241,2024-07-18,716.814
